In [1]:
import torch
import torch.nn as nn
import time
import os

## Load model

In [2]:
class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FCN, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out) 
        return out

In [3]:
model = FCN(1,1,1)
model = torch.load('model.pth')
model

FCN(
  (l1): Linear(in_features=36, out_features=20, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=20, out_features=4, bias=True)
)

In [4]:
for weight in model.state_dict():
    print(weight)

l1.weight
l1.bias
l2.weight
l2.bias


In [5]:
for key, val in model.state_dict().items():
    print(key, val.shape)

l1.weight torch.Size([20, 36])
l1.bias torch.Size([20])
l2.weight torch.Size([4, 20])
l2.bias torch.Size([4])


## Print weights for HLS

In [6]:
l1_weight = model.state_dict()['l1.weight'].tolist()
l2_weight = model.state_dict()['l2.weight'].tolist()
l1_bias = model.state_dict()['l1.bias'].tolist()
l2_bias = model.state_dict()['l2.bias'].tolist()

In [7]:
print("const float l1_bias[HIDDEN_SIZE] = {", end='')
biases = ''
for b in l1_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l1_bias[HIDDEN_SIZE] = {-0.05334458872675896, -0.13644450902938843, -0.2673816680908203, 0.12261010706424713, -0.17974506318569183, -0.008566997945308685, 0.1170906275510788, 0.005460590589791536, -0.14909523725509644, 0.16343431174755096, 0.3446892201900482, -0.08835111558437347, -0.019531795755028725, 0.025058090686798096, 0.5078228712081909, 0.06073518097400665, 0.04052054509520531, -0.17416788637638092, 0.0005443513509817421, -0.1994025558233261};


In [8]:
print("const float l2_bias[NUM_CLASSES] = {", end='')
biases = ''
for b in l2_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l2_bias[NUM_CLASSES] = {0.011831509880721569, -0.2243700921535492, 0.0035149359609931707, 0.4138830602169037};


In [9]:
print("const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {", end='')
weights = ''
for row in l1_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {{-0.06309240311384201, -0.2532886862754822, 1.0538132190704346, 0.21994014084339142, 0.003935294225811958, 0.38774168491363525, -0.5814903378486633, 0.17311634123325348, -0.3715142011642456, -0.7375488877296448, -0.455607533454895, -0.8833975791931152, 0.8786311149597168, -0.9904577136039734, 0.30869221687316895, 0.9575240612030029, 0.8208673596382141, 0.8438540101051331, 0.008634628728032112, -0.11326632648706436, -0.1425938457250595, 0.047284919768571854, 0.2720261216163635, 0.16587021946907043, 0.0936233252286911, 0.09328111261129379, -0.13498376309871674, -0.1771565079689026, -0.0023882025852799416, 0.20226018130779266, 0.026079652830958366, 0.11071846634149551, 0.0380258709192276, -0.20030444860458374, -0.07657022029161453, -0.2014259248971939},
{-0.10682124644517899, 0.03815050795674324, -0.12337307631969452, -0.050704747438430786, -0.13342903554439545, 0.0677933394908905, -0.17710669338703156, 0.11545603722333908, -0.21886453032

In [10]:
print("const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {", end='')
weights = ''
for row in l2_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {{-0.1280953586101532, -0.12167757004499435, 0.010786538943648338, -0.18267615139484406, -0.14638663828372955, 0.17025969922542572, 0.047933101654052734, -0.09269899129867554, -0.35289397835731506, -0.21688753366470337, -0.13649290800094604, 0.04241060093045235, 0.07317662984132767, 0.10657131671905518, -0.09870198369026184, 0.1900177299976349, 0.22705604135990143, 0.23391780257225037, -0.12960085272789001, -0.054554954171180725},
{0.11042877286672592, 0.0495004840195179, 0.1541384905576706, 0.1309833526611328, 0.10469444841146469, 0.02568761073052883, -0.16886113584041595, -0.02422128990292549, 0.1624232679605484, -0.01301911473274231, -0.28847455978393555, -0.1841299831867218, -0.10822442173957825, 0.2215580940246582, -0.2397242784500122, -0.15751749277114868, 0.04291843622922897, 0.0031323463190346956, 0.04515921324491501, -0.09335663914680481},
{0.06692615896463394, -0.032778121531009674, -0.19652682542800903, -0.11436666548252106,

## Get test data for software-hardware comparison

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [33]:
DATA_DIR = 'data'
RAW = DATA_DIR + '/raw'
NUM_FEATURES = 36

In [34]:
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
X_test = test_df.drop('action', axis='columns')
y_test = test_df['action']

In [36]:
data = torch.Tensor(X_test.iloc[0].astype(float))

In [37]:
data

tensor([ 4.4000e-01, -9.3000e-01, -2.0600e-02,  2.4825e-01,  1.1750e-01,
         1.6324e-01,  2.3500e+00, -1.0500e+00,  4.6900e-01,  6.2036e-01,
         8.0750e-01,  5.0636e-01,  4.6000e-01, -2.0700e+00, -4.0840e-01,
         6.2564e-01,  7.9250e-01,  5.1262e-01,  1.8701e+02, -3.1759e+02,
        -1.2022e+01,  7.1242e+01,  1.1445e+01,  3.9310e+01,  2.7191e+02,
        -2.7341e+02,  1.3346e+00,  1.0965e+02,  2.3070e+01,  6.3755e+01,
         2.0676e+02, -1.9849e+02, -2.3408e+00,  7.3921e+01,  9.9550e+00,
         4.1250e+01])

In [38]:
with torch.no_grad():
    start_time = time.time()
    output = model(data)
    action = np.argmax(output)
    print(f"Inference time: {time.time() - start_time}")    
    print(action)

Inference time: 0.0133819580078125
tensor(2)


In [39]:
y_test[0]

2

## Compare software-hardware running time and accuracy

In [40]:
# 1 sample

with torch.no_grad():
    start_time = time.time()

    data = torch.Tensor(X_test.iloc[0].astype(float))
    output = model(data)
    action = np.argmax(output)

    end_time = time.time()
    print(f"Prediction: {action}; Ground truth value: {y_test[0]}")
    print(f"Inference time: {((end_time - start_time) * 1000):.4f} ms") 

Prediction: 2; Ground truth value: 2
Inference time: 3.7529 ms


In [41]:
# whole test set (or dev + test set used in training)

with torch.no_grad():
    start_time = time.time()
    
    num_correct = 0
    num_samples = 0

    for i in range(len(X_test[:])):
        data = torch.Tensor(X_test.iloc[i].astype(float))
        output = model(data)
        action = np.argmax(output)
        num_samples += 1
        num_correct += (action == y_test[i])

    end_time = time.time()
    print(f"Accuracy: {(100.0 * num_correct / num_samples):.2f}%")
    print(f"Inference time: {(end_time - start_time):.4f} s")    

Accuracy: 96.88%
Inference time: 0.0195 s


In [42]:
len(X_test)

32